# Aplicação de Métodos de Clustering na Análise do Diagrama de Hertzsprung-Russell
## Por Marcella Decembrino e Marcelo Lopes

### Tópicos
- Setup
- Seleção do Dataset
- Implementação de Técnicas de Clustering
- Análise de Dados - Clustering por HDBSCAN
- Construção do Diagrama HR
- Limitação de Distância e Viés de Tipo Estelar
- Conclusões
- Referências

### Seleção do dataset
Neste trabalho foi utilizado um dataset com dados reais provenientes do HYG Stellar Database, um catálogo astronômico que reúne informações das bases Hipparcos, Yale Bright Star Catalog e Gliese.

Diversas informações sobre estrelas observadas são fornecidas, incluindo:
- Magnitude aparente (`mag`) e magnitude absoluta (`absmag`)
- Distância (`dist`, em parsecs)
- Índice de cor (`ci`, color index B-V)
- Temperatura (`temp`, em Kelvin)
- Luminosidade (`lum`, em múltiplos da luminosidade do Sol)
- Coordenadas espaciais (`x`, `y`, `z`) e coordenadas astronômicas (`ra`, `dec`)

Assim como as colunas `proper`, `con`, `var`, `var_min` e `var_max`, representando nomes, constelações e outros parâmetros fotométricos que não serão utilizados.

Visto que o conjunto de dados é proveniente de observações reais, a limpeza e análise de dados deverá considerar incertezas observacionais, ruídos instrumentais e metodológicos, assim como possíveis vieses provientes de limitações físicas dos métodos.

Considerando tais limitações, o objetivo é encontrar agrupamentos de estrelas que possuam características semelhantes, permitindo posteriormente comparações e o estudo da evolução estelar.

# Setup

Nesta seção, são carregadas bibliotecas para a obtenção, tratamento e processamento dos dados, visualização exploratória e aplicação de algoritmos de clustering.

In [ ]:
# Obtenção, tratamento e processamento dos dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import altair as alt

# Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import hdbscan

In [ ]:
alt.data_transformers.disable_max_rows() # permitir a utilização de uma grande quantidade de dados pelo Altair

# Seleção do Dataset

Primeiramente, serão definidos parâmetros identificando o nome do arquivo e as principais colunas a serem utilizadas para as análises, assim como colunas nunca utilizadas, que serão removidas.

In [ ]:
# Definição do nome do arquivo correspondente ao dataset
dataset_file = "stars.csv"

# Definição de variáveis para os nomes das principais colunas a serem utilizadas
# Facilita acesso posterior
col_color = "ci"
col_mag = "absmag"
col_temp = "temp"
col_lum = "lum"

# Definição das colunas a serem removidas
cols_remover = ["proper", "con", "var", "var_min", "var_max"]

A seguir é realizado o carregamento do dataset em formato CSV por meio da biblioteca pandas. Após o carregamento, são removidas as colunas não utilizadas, e é feita uma inspeção para verificar as dimensões, tipos de dados e possíveis inconsistências na importação.

In [ ]:
# Leitura do arquivo e filtro inicial

leitura = pd.read_csv(dataset_file) # leitura do arquivo
df = leitura.copy().drop(columns=cols_remover, errors="ignore") # remoção de colunas não usadas

# Inspeção do dataset

print("Dimensões do dataset:", df.shape) # verificação do número de colunas e de dados

df.info()
df.head()

Antes de realizar limpeza e análise, os dados são visualizados por meio do diagrama Hertzsprung-Russell (HR).

Os objetivos desta etapa são:
- Identificar possíveis outliers
- Verificar consistência das variáveis

O diagrama relaciona índice de cor, uma variável que representa a diferença entre as intensidades de luz azul e vermelha observadas, com magnitude absoluta, relacionada à luminosidade intrínseca de uma estrela. 

É esperado que populações de estrelas semelhantes ocupem regiões específicas do plano do diagrama HR, gerando agrupamentos naturais.

In [ ]:
# Visualização inicial dos dados importados
# Uso de matplotlib para facilitar a visualização de um volume grande de dados

plt.style.use("default") # garantir que o fundo será branco (evita interferência de outros gráficos mais abaixo)

plt.figure(figsize=(8, 6)) # definição das dimensões da figura

plt.scatter(df[col_color], df[col_mag], c='blue', s=2) # plot de mag abs por index de cor

plt.gca().invert_yaxis() # inversão do eixo y para seguir o padrão do diagrama HR

# Definição do nome dos eixos e do título do gráfico
plt.xlabel("Color Index (ci)")
plt.ylabel("Magnitude Absoluta")
plt.title("Diagrama HR - Dados Importados")

plt.show() # exibição do gráfico

No diagrama gerado acima, é possível observar uma grande presença de estrelas na faixa de luminosidade absoluta entre -10 e -15. Embora existam estrelas com tal luminosidade, elas são extremamente raras na Via Láctea. 

A presença de uma grande quantidade de estrelas deste tipo sugere possíveis inconsistências nos dados, como vieses observacionais ou parâmetros pouco precisos gerando valores extremos para a magnitude absoluta por propagação de incertezas.

Para investigação, utiliza-se o método `.describe()` para analisar uma forma resumida das estatísticas do dataset.

In [ ]:
# Exibição das principais informações do dataset para inspeção

df.describe()

O primeiro erro notável é em relação às distâncias. O valor máximo para elas está extremamante alto, e medidas astronômicas à tais distâncias podem ser extremamente imprecisas, por possíveis efeitos de nuvens de poeira, erros de fotometria, entre outros. 

É possível observar que o valor máximo para as distâncias é extremamente elevado. Medidas astronômicas para tais valores estão fortemente sujeitas a incertezas fotométricas, efeitos de absorção por poeira interestelar, além de limitações instrumentais e erros na medição de paralaxe.

Portanto, para a primeira limpeza, optou-se pela remoção de estrelas com distância acima de 5000 parsecs.

In [ ]:
# Filtro de estrelas muito distantes

df = df[df["dist"] < 5000]

In [ ]:
# Visualização dos dados após a primeira limpeza

plt.style.use("default") # garantir que o fundo será branco (evita interferência de outros gráficos mais abaixo)

plt.figure(figsize=(8, 6)) # definição das dimensões da figura

plt.scatter(df[col_color], df[col_mag], c='blue', s=2) # plot de mag abs por index de cor

plt.gca().invert_yaxis() # inversão do eixo y para seguir o padrão do diagrama HR

# Definição do nome dos eixos e do título do gráfico
plt.xlabel("Color Index (ci)")
plt.ylabel("Magnitude Absoluta")
plt.title("Diagrama HR - Dados Após Primeira Limpeza")

plt.show() # exibição do gráfico

Após a primeira etapa, observa-se que o diagrama acima apresenta uma distribuição mais coerente com o esperado para populações estelares reais. A concentração de estrelas extremamente luminosas no diagrama foi reduzida de forma significativa somente utilizando o filtro de distâncias. Porém, ainda é possível observar certos valores extremos, especialmente em relação ao Color Index.

A seguir, são implementados os seguintes filtros, a fim de garantir a consistência física e estabilidade dos modelos de clustering a serem construídos posteriormente:
- Color Index: entre -0.5 e 2.5 (a fim de evitar desvios para o vermelho acentuados)
- Magnitude Absoluta: entre -15 e 20 (valores esperados para estrelas)
- Temperatura: entre 2000 K e 50000 K (valores esperados para estrelas)
- Luminosidade: acima de 0 (a luminosidade é sempre positiva)

Os intervalos acima abrangem praticamente todas as estrelas conhecidas, excluindo somente valores inconsistentes e resultantes de erros experimentais.

Além disso, valores extremos como $-\infty$ e $\infty$ serão substituídos por `NaN`, e entradas com valores `NaN` nas principais colunas serão removidas do dataframe.

In [ ]:
# Segunda limpeza dos dados

df = df.replace([np.inf, -np.inf], np.nan) # substituição de valores extremos por NaN
df = df.dropna(subset=[col_color, col_mag, col_temp, col_lum]) # remoção de entradas com valores NaN nas principais colunas

# Filtros para consistência e estabilidade dos modelos de clustering
df = df[df[col_color].between(-0.5, 2.5)]
df = df[df[col_mag].between(-15, 20)]
df = df[df[col_temp].between(2000, 50000)]
df = df[df[col_lum] > 0]

In [ ]:
# Visualização dos dados após a segunda limpeza

plt.style.use("default") # garantir que o fundo será branco (evita interferência de outros gráficos mais abaixo)

plt.figure(figsize=(8, 6)) # definição das dimensões da figura

plt.scatter(df[col_color], df[col_mag], c='blue', s=2) # plot de mag abs por index de cor

plt.gca().invert_yaxis() # inversão do eixo y para seguir o padrão do diagrama HR

# Definição do nome dos eixos e do título do gráfico
plt.xlabel("Color Index (ci)")
plt.ylabel("Magnitude Absoluta")
plt.title("Diagrama HR - Dados Após Segunda Limpeza")

plt.show() # exibição do gráfico

Após a segunda etapa de limpeza, observa-se que o diagrama possui uma distribuição compatível com a estrutura conhecida para populações estelares. 

Em seguida, calcula-se um raio estimado para cada estrela, com base na Lei de Stefan-Boltzmann ($L=4\pi R^2 T^4$). A partir disso, é possível derivar uma fórmula para isolar o raio em unidades solares. Considerando $T_\odot=5772 K$, tem-se que:

$$\frac{L}{L_\odot}=\left(\frac{R}{R_\odot}\right)^2\left(\frac{T}{T_\odot}\right)^4$$

$$\frac{R}{R_\odot}=\sqrt{\frac{L}{(T/T_\odot)^4}}$$

Além do raio estimado, será definida uma variável indicando as estrelas com características compatíveis com a categoria de "anãs brancas" (quentes e pouco luminosas). 

Essas estrelas foram selecionadas como as que possuem magnitude absoluta acima de $10$ e índice de cor abaixo de $0.5$.

Abaixo, serão calculados o raio estimado para cada estrela, e uma "flag", indicando a faixa de magnitude e cor na qual as estrelas provavelmente são do tipo "anã branca".

In [ ]:
# Criação da coluna de raios em unidades solares

Tsun = 5772 # temperatura do sol
df["radius_est"] = np.sqrt(df[col_lum] / ((df[col_temp] / Tsun) ** 4)) # aplicação da fórmula
col_raio = "radius_est" # variável referente ao nome da coluna

# Identificação de anãs brancas

# Criação de uma coluna que identifica possíveis anãs brancas
df["white_dwarf_flag"] = (
    (df[col_mag] > 10) & # aplicação dos filtros citados acima
    (df[col_color] < 0.5)
)

Antes das análises de clusterização, seleciona-se uma amostra do dataframe previamente filtrado, considerando 50.000 estrelas. Essa etapa reduz o custo computacional dos algoritmos, mas mantém a representatividade estatística da distribuição, além de garantir a reprodutibilidade por meio do estado aleatório `random_state=56`.

In [ ]:
# Seleção de 50.000 dados para as análises das seções seguintes

df = df.sample(50000, random_state=56)

Abaixo, são determinadas as cores e classes espectrais de cada estrela, com base em faixas de temperatura.

Além disso, adiciona-se uma coluna com o logaritmo da temperatura na base 10, garantindo consistência com o diagrama HR clássico, além de reduzir assimetrias na distribuição, auxiliando na aplicação dos algoritmos de clustering.

In [ ]:
# Definição das classes espectrais

classes = [0, 3700, 5200, 6000, 7500, 10000, 30000, np.inf] # valores limites de cada classe
labels = ["M", "K", "G", "F", "A", "B", "O"] # nome de cada classe

# Cores que se referem a cada classe para futuros plots
cores_espectrais = {
    "O": "#4f6cff",
    "B": "#7ea8ff",
    "A": "#dbe9ff",
    "F": "#fff7d6",
    "G": "#ffe86b",
    "K": "#ffb347",
    "M": "#ff5c5c"
}

# Criação de uma coluna que associa a cada entrada, sua classe espectral
df["classe_espectral"] = pd.cut(df[col_temp], bins=classes, labels=labels, right=False)

# Criação da coluna de log(temp) de cada estrela

df["log_temp"] = np.log10(df["temp"]) # criação da coluna
col_temp_log = "log_temp" # criação da variável referente ao nome da coluna

O diagrama HR da amostra, apresentado abaixo, demonstra a conservação da estrutura esperada para populações estelares. É possível identificar regiões bem conhecidas, como a sequência principal, a região das gigantes e uma concentração inferior de objetos quentes e de baixa luminosidade, compatível com as anãs brancas.

Posteriormente, será avaliado como os algoritmos de clusterização se adequam a tais grupos, ou seja, se foi possível identificar automaticamente cada população como um grupo distinto.

In [ ]:
# Diagrama HR com os dados filtrados

plt.style.use("default") # garantir que o fundo será branco (evita interferência de outros gráficos mais abaixo)

plt.figure(figsize=(8, 6)) # definição das dimensões da figura

# Plot das estrelas não classificadas como possíveis anãs brancas

nao_wd = df[~df["white_dwarf_flag"]] # seleção das não classificadas como anãs brancas
plt.scatter(nao_wd[col_color], nao_wd[col_mag], c='blue', s=2) # plot em azul

# Plot das possíveis anãs brancas

wd = df[df["white_dwarf_flag"]] # seleção dos dados referentes a anãs brancas
plt.scatter(wd[col_color], wd[col_mag], 
color = "red", s=2, label="Possíveis anãs brancas") # plot em vermelho

# Inversão do eixo y
plt.gca().invert_yaxis()

# Definição do nome dos eixos e do título do gráfico
plt.xlabel("Color Index (ci)")
plt.ylabel("Magnitude Absoluta")
plt.title("Diagrama HR - Dados da Amostra")

# Definição da legenda
plt.legend(markerscale=1, fontsize="small", loc="best") # fonte pequena com posição escolhida pelo matplotlib

plt.show() # exibição do gráfico


# Implementação de Técnicas de Clustering

## Configurações Iniciais

Para a aplicação dos algoritmos de clustering e exibição final, selecionaram-se duas variáveis fundamentais para o diagrama HR:
- Magnitude Absoluta (representada por `col_mag`)
- Logaritmo da Temperatura em base 10 (representado por `col_temp_log`)

A escolha destas variáveis é justificada pelo fato de que o diagrama HR permite distinguir naturalmente grupos de estrelas nas regiões mencionadas durante a importação de dados. 

Para fins de visualização e facilidade de interpretação no diagrama final, o logaritmo da temperatura é utilizado para substituir o índice de cores, mas é importante ressaltar que ambos estão fortemente correlacionados, e frequentemente a temperatura é estimada com base no índice de cores após uma série de correções fotométricas.

Utiliza-se o logaritmo da temperatura no eixo horizontal para reduzir o impacto de assimetrias, dada a grande variabilidade entre estrelas.

Além disso, antes da aplicação dos algoritmos, utiliza-se o `StandardScaler`, transformando as variáveis de modo que a média seja $0$ e o desvio padrão seja $1$, ou seja, padronizando por meio do Z-score. Esse passo é necessário para garantir que magnitude e temperatura contribuam de forma equilibrada, pois certos algoritmos (por exemplo, K-Means) são sensíveis à escala das variáveis, podendo causar distorções no cálculo quando as variáveis possuem ordens de grandeza e variância muito distintas.

In [ ]:
# Seleção das variáveis Magnitude Abs. e Log da Temp. para clustering
variaveis = df[[col_mag, col_temp_log]]

# Obtenção do Z-score das variáveis
scaler = StandardScaler() # criação da instância que irá definir a nova escala das variáveis
variaveis_s = scaler.fit_transform(variaveis) # aplicação no conjunto de variáveis

Em seguida, é definida uma função responsável por plotar o diagrama HR colorido de acordo com a clusterização obtida.

A fim de manter o formato padronizado para o diagrama HR, os eixos X e Y são invertidos, garantindo que temperaturas maiores estejam à esquerda e estrelas mais luminosas (ou seja, com magnitude absoluta menor) estejam na parte superior do gráfico. Além disso, define-se uma escala logarítmica para o eixo da temperatura.

In [ ]:
# Função para plotar os dados a serem coloridos de acordo com os clusters

def plot_cluster (dados_x, dados_y, clusters, metodo):

    plt.style.use("default") # garantir que o fundo do gráfico não seja preto
    
    plt.figure(figsize=(8, 6)) # definição das dimensões da figura

    # Plot dos dados originais coloridos com base no cluster a que pertencem
    for c in np.unique(clusters): # um loop para cada cluster
        mask = clusters == c # máscara booleana para plotar os dados apenas do cluster atual
        plt.scatter(dados_x[mask], dados_y[mask], # seleção dos elementos onde a máscara é 'True'
                    s=2, alpha=1,
                    label=f"Cluster {c}") # rótulo do cluster atual

    # Ajuste dos eixos

    # Inversão dos eixos x (temperatura) e y (magnitude abs) para corresponder ao diagrama
    plt.gca().invert_yaxis() # inverter eixo y
    plt.gca().invert_xaxis() # inverter eixo x

    plt.xscale("log") # eixo x na escala log para melhor visualização

    plt.xlabel("Temperatura (K)") # eixo x equivalente ao eixo da temperatura
    plt.ylabel("Magnitude Absoluta") # eixo y equivalente ao eixo da magnitude absoluta

    # Ajuste do título e da legenda

    plt.title(f"Diagrama HR com Clustering por {metodo}") # título adaptado ao método de clustering utilizado
    plt.legend(markerscale=1, fontsize="small", loc="best") # fonte pequena com posição escolhida pelo matplotlib

    # Exibição do gráfico

    plt.show()

    return # não retorna nada para apenas exibir o gráfico no notebook

## K-Means

O primeiro algoritmo a ser utilizado será o K-Means, por sua simplicidade, ampla utilização e eficiência computacional.

O método é baseado na minimização do quadrado das distâncias entre cada ponto e o centróide de seu respectivo cluster. Assim, busca-se minimizar a "inércia", definida como a soma de todas estas distâncias para cada cluster.

A fim de encontrar o valor ideal para o número de clusters ($k$), que deve ser definido previamente, foram implementados dois métodos para auxiliar na escolha:
- Elbow Method
- Silhouette Score

O "Elbow Method", ou "Método do Cotovelo" analisa o comportamento da inércia conforme o número de clusters aumenta. A medida que $k$ cresce, a inércia tende a diminuir, pois os clusters se tornam menores e mais específicos. Porém, após um certo ponto, a redução passa a ser menos significativa. Esse ponto é definido como o "cotovelo", e indica um candidato adequado para o valor de $k$, equilibrando quantidade de clusters e qualidade da separação.

Já o "Silhouette score" avalia a qualidade da clusterização considerando a coesão interna dos grupos e separação entre eles. O score varia entre -1 e 1, sendo que valores próximos de 1 indicam separação forte, valores próximos de 0 indicam sobreposição, e valores negativos sugerem uma classificação incorreta. Esse método avalia não somente a compactação dos grupos, mas também permite uma visão da estrutura geral.

In [ ]:
def encontrar_k (dados, k_max): # k_max: maior k a ser incluído na visualização

    plt.style.use("default") # garantir fundo branco

    k_elbow = [] # valores de k para cada valor de inércia correspondente
    k_silhouettes = [] # valores de k para cada silhouette score correspondente
    inertias = [] # valores encontrados para a inércia
    silhouettes = [] # valores encontrados para os silhouette scores

    # Gera silhouette scores e valores de inércia para k de 1 (inércia) ou 2 (silhouette) a k_max
    for k in range (1, k_max + 1): 
        kmeans = KMeans(n_clusters=k, random_state=56) # inicialização da instância de KMeans
        kmeans.fit(dados) # aplicação do processo ao conjunto de dados
        
        k_elbow.append(k) # armazenamento do valor de k atual
        inertias.append(kmeans.inertia_) # cálculo e armazenamento do valor de inércia para o k atual

        if k != 1: # Silhouette Scores só são válidos para k > 1
            score = silhouette_score(dados, kmeans.labels_) # cálculo do silhouette score
            silhouettes.append(score) # armazenamento do silhouette score
            k_silhouettes.append(k) # armazenamento do valor de k atual
    
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5)) # criação da figura e dos dois subplots

    # Gráfico do elbow method
    axs[0].plot(k_elbow, inertias, marker="o", linestyle="-")
    axs[0].set_title("Elbow Method")
    axs[0].set_ylabel("Inércia")
    axs[0].set_xlabel("k")

    # Gráfico de silhouette score
    axs[1].plot(k_silhouettes, silhouettes, marker="o", linestyle="-")
    axs[1].set_title("Silhouette Score")
    axs[1].set_ylabel("Score")
    axs[1].set_xlabel("k")

    plt.tight_layout() # reajuste dos subplots para melhor organização dos gráficos
    plt.show() # exibição do gráfico

    return

Nessa etapa, o algoritmo é aplicado ao espaço bidimensional formado pela magnitude e logaritmo da temperatura, que definem o plano do diagrama HR, a fim de investigar se o algoritmo identifica agrupamentos naturais de estrelas.

Primeiramente, observaremos os valores sugeridos para $k$, por meio dos gráficos gerados pela função `encontrar_k`.

In [ ]:
# Exibição dos gráficos de Elbow Method e de Silhouette Score para variaveis_s

encontrar_k(variaveis_s, 5)

Com base na análise acima, observa-se que os valores de $k=3$ e $k=4$ são fortes candidatos para o K-Means, com $k=4$ sendo ligeiramente superior para Silhouette Score.

A primeira análise será realizada considerando $k=3$, e gerando um gráfico contendo as estrelas e sua separação em clusters, identificada pelas cores.

In [ ]:
# Aplicação do modelo de clustering de K-Means com k = 3

kmeans_3 = KMeans(n_clusters=3, random_state=50) # geração da instância de KMeans referente a k = 3
kmeans_3.fit(variaveis_s) # aplicação da instância aos dados padronizados

plot_cluster (df[col_temp], df[col_mag], kmeans_3.labels_, "K-Means (k = 3)") # visualização dos clusters

Após a aplicação do K-Means com $k=3$, observa-se uma segmentação do diagrama em três regiões bem definidas. É possível identificar uma região composta por estrelas de baixa temperatura e alta luminosidade, uma região com estrelas de maior temperatura e luminosidade semelhante, e outra para estrelas com luminosidade menor.

A seguir, o algoritmo é aplicado para $k=4$.

In [ ]:
# Aplicação do modelo de clustering de K-Means com k = 4

kmeans_4 = KMeans(n_clusters=4, random_state=50) # geração da instância de KMeans referente a k = 4
kmeans_4.fit(variaveis_s) # aplicação da instância aos dados padronizados

plot_cluster (df[col_temp], df[col_mag], kmeans_4.labels_, "K-Means (k = 4)") # visualização dos clusters

Com $k=4$, o algoritmo mantém a estrutura geral semelhante ao caso $k=3$, mas subdivide a região superior direita em duas partes.

Adicionalmente, será analisado também o caso $k=2$.

In [ ]:
# Aplicação do modelo de clustering de K-Means com k = 2

kmeans_2 = KMeans(n_clusters=2, random_state=50) # geração da instância de KMeans referente a k = 2
kmeans_2.fit(variaveis_s) # aplicação da instância aos dados padronizados

plot_cluster (df[col_temp], df[col_mag], kmeans_2.labels_, "K-Means (k = 2)") # visualização dos clusters

Para o caso $k=2$, observa-se uma divisão clara entre a região superior direita e as demais regiões do diagrama.

Assim, observa-se que o algoritmo é capaz de capturar diferenças no diagrama HR, especialmente na separação da região superior direita, classificada teoricamente como a região das gigantes.

Porém, a depender do valor utilizado para $k$, também surgem subdivisões internas nos grupos, especialmente na região da sequência principal. Além disso, o algoritmo não é capaz de identificar as anãs brancas, o que pode ser também atribuído à baixa representatividade da população de tais estrelas.

Em geral, a clusterização realizada pelo K-Means é capaz de identificar algumas regiões, mas tende a criar subdivisões adicionais nas regiões, e não consegue identificar regiões com densidade menor, como as anãs brancas, nem distinguir ruídos.

## DBSCAN

Diferentemente do K-Means, o DBSCAN é um algoritmo que baseia-se em densidade, sendo capaz de identificar reigões densas no espaço sem a deinição prévia específica do número de clusters. Além disso, um ponto positivo para dados astronômicos é a capacidade de classificação como ruído.

Diversos valores para os parâmetros `eps` e `min_samples` foram testadas. A configuração adotada foi a que apresentou o melhor equilíbrio visual, identificando de forma contínua a sequência principal, sem a presença de subdivisões artificiais, e separando a região das gigantes vermelhas de forma adequada.

Apesar disso, na visualização abaixo, é possível observar que as anãs brancas ainda são classificadas como ruído, o que novamente pode ser atribuído à baixa densidade dessa população no dataset utilizado, que será explorada posteriormente em detalhes.

In [ ]:
# Aplicação do modelo de clustering de DBSCAN

dbscan_modelo = DBSCAN(eps=0.1, min_samples=133).fit(variaveis_s) # criação do modelo para epsilon = 0.1 e min_samples = 133
dbscan_labels = dbscan_modelo.labels_ # obtenção da classificação dos dados o modelo de dbscan gerado

plot_cluster (df[col_temp], df[col_mag], dbscan_labels, "DBSCAN") # visualização dos clusters

## GMM

O algoritmo GMM (Gaussian Mixture Model) se baseia na suposição de que os dados são gerados por uma combinação de distribuições normais. Assim, diferentemente do K-Means, o GMM modela cada cluster como uma distribuição normal própria, com seus parâmetros individuais.

Os parâmetros são ajustados por um algoritmo chamado "Expectation-Maximization". Esse algoritmo calcula, para cada ponto, uma probabilidade de pertencer a uma clusterização aleatória inicial. Então, buscando maximizar as probabilidades, o algoritmo cria novas clusterizações a cada passo, até convergir.

A fim de analisar o desempenho do modelo, ele será aplicado para os casos com 2, 3 e 4 clusters a seguir, começando com 2 clusters.

In [ ]:
# Aplicação do modelo de clustering de GMM para 2 clusters

# Criação do modelo
gmm = GaussianMixture(
    n_components=2, # divisão dos dados em duas populações
    covariance_type='full', # adequação à inclinação da sequência principal
    random_state=50)

# Aplicação do modelo às variáveis
gmm_labels_2 = gmm.fit_predict(variaveis_s) 

# Visualização dos clusters
plot_cluster (df[col_temp], df[col_mag], gmm_labels_2, "GMM")

Com dois clusters, o modelo separa a região das gigantes do restante das estrelas, possuindo uma divisão semelhante à obtida pelo K-Means, mas com fronteiras suavizadas.

In [ ]:
# Aplicação do modelo de clustering de GMM para 3 clusters

# Criação do modelo
gmm = GaussianMixture(
    n_components=3, # divisão dos dados em duas populações
    covariance_type='full', # adequação à inclinação da sequência principal
    random_state=50)

# Aplicação do modelo às variáveis
gmm_labels_3 = gmm.fit_predict(variaveis_s) 

# Visualização dos clusters
plot_cluster (df[col_temp], df[col_mag], gmm_labels_3, "GMM")

Para três clusters, o modelo agora captura as anãs brancas juntamente com as gigantes. Isso ocorre pois, como a grande maioria das estrelas pertence à sequência principal, o modelo utilizou dois clusters para elas, maximizando a probabilidade, de modo que o restante do espaço não impactou de forma tão significativa, por sua baixa representatividade.

In [ ]:
# Aplicação do modelo de clustering de GMM para 4 clusters

# Criação do modelo
gmm = GaussianMixture(
    n_components=4, # divisão dos dados em duas populações
    covariance_type='full', # adequação à inclinação da sequência principal
    random_state=50)

# Aplicação do modelo às variáveis
gmm_labels_4 = gmm.fit_predict(variaveis_s) 

# Visualização dos clusters
plot_cluster (df[col_temp], df[col_mag], gmm_labels_4, "GMM")

Para quatro clusters, o GMM subdividiu a região das gigantes em duas componentes distintas, assim como a sequência principal, e a região das anãs brancas foi absorvida pela parte superior da sequência principal.

Assim, verifica-se que a modelagem com base em distribuições normais não reflete necessariamente a separação física real, refletindo na imprecisão obtida pelo modelo.

## HDBSCAN

O HDBSCAN pode ser interpretado como uma extensão do DBSCAN, se diferenciando por lidar melhor com variações de densidade entre os grupos.

O parâmetro `min_samples` é o mínimo de pontos nos arredores de um ponto para que este seja considerado um "core point". Já o parâmetro `min_cluster_size` estabelece a quantidade mínima de pontos para que um cluster seja considerado válido. Por padrão, a função assume `min_cluster_size = min_samples`.

Para os dados aqui analisados, essa característica pode auxiliar a identificar grupos menos densos, por isso, o algoritmo do HDBSCAN também será utilizado.

In [ ]:
# Aplicação do modelo de clustering de HDBSCAN

min_cluster_size = 200 # tamanho mínimo dos clusters p/ manter estruturas maiores
cluster_hdbscan = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size) # aplicação do modelo

df["hdbscan"] = cluster_hdbscan.fit_predict(variaveis_s) # armazenamento das classificações

plot_cluster (df[col_temp], df[col_mag], df["hdbscan"], "HDBSCAN") # visualização dos clusters

Como observado no diagrama acima, o HDBSCAN foi capaz de identificar clusters associados à sequência principal e à região das gigantes, mas também classificou as anãs brancas como ruído, assim como o DBSCAN. Apesar das semelhanças, o HDBSCAN também foi capaz de captar a região mais fina da sequência principal, ainda que não tenha sido completamente.

In [ ]:
# Aplicação do modelo de clustering de HDBSCAN (tentativa 02)

min_cluster_size = 30 # redução do tamanho mínimo para captar as anãs brancas
cluster_hdbscan = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, 
                    min_samples = 5) # redução da 'densidade mínima' para captar as anãs brancas

plot_cluster (df[col_temp], df[col_mag], 
    cluster_hdbscan.fit_predict(variaveis_s), "HDBSCAN") # visualização dos clusters

Para a segunda tentativa, foi utilizado um valor menor para o tamanho mínimo dos clusters e a densidade mínima dos pontos a fim de tentar captar as anãs brancas. Porém, apesar da clusterização ter identificado as anãs brancas, não sucedeu na distinção entre sequência principal e gigantes, por conta da proximidade entre os grupos.

## Conclusões

Considerando os resultados obtidos para os quatro algoritmos (K-Means, DBSCAN, GMM e HDBSCAN), foi escolhido o HDBSCAN para analisar os resultados derivados da clusterização obtida.

Os algoritmos K-Means e GMM não diferenciam ruído, o que é negativo para dados astronômicos, que frequentemente possuem incertezas altas, e como a densidade dos agrupamentos de estrelas não é constante, opta-se pelo HDBSCAN, por sua capacidade em lidar melhor com tais tipos de dados.

# Análise de Dados - Clustering por HDBSCAN

Para as análises finais, o HDBSCAN por sua maior flexibilidade em lidar com densidade variável e ruído, características presentes no conjunto de dados estelares.

In [ ]:
# Plot do modelo escolhido (DBSCAN com min_cluster_size = min_samples = 200)

plot_cluster (df[col_temp], df[col_mag], df["hdbscan"], "HDBSCAN")

Observou-se que o HDBSCAN classificou a maioria das candidatas a anãs brancas como ruído, em razão de sua baixa densidade. Considerando a boa definição dessa região no diagrama HR, optou-se por analisá-la separadamente, atribuindo os pontos presentes nela a um cluster específico, para fins de comparação.

Abaixo, a fim de analisar e comparar as características dos clusters, definimos o dataset `df_sem_ruido`, o qual contém apenas pontos pertencentes aos clusters não classificados como ruído.

In [ ]:
# Alteração dos clusters com a classificação astrofísica de anãs brancas

df["clusters"] = df["hdbscan"] # criação de uma nova coluna para a nova divisão de clusters
df.loc[df["white_dwarf_flag"] == True, "clusters"] = 2 # definição das anãs brancas como cluster 2

df_sem_ruido = df[df["clusters"] != -1].copy() # criação de um df sem ruído para as análises a seguir

In [ ]:
# Identificação dos clusters resultantes

df_sem_ruido["clusters"].unique()

Abaixo, apresenta-se o diagrama HR com os agrupamentos definidos pelo HDBSCAN, incluindo as candidatas a anãs brancas destacadas.

In [ ]:
# Plot dos três clusters resultantes

plot_cluster (df[col_temp], df[col_mag], df["clusters"], "HDBSCAN (alterado)")

## Análises por Cluster

Nesta subseção, será analisada a distribuição das classes espectrais dentro de cada cluster, com a finalidade de avaliar a coerência física dos clusters obtidos.

O histograma gerado abaixo apresenta a composição percentual das classes espectrais em cada cluster, permitindo uma verificação das tendências físicas esperadas no diagrama HR.

In [ ]:
# Definição de ordem para as classes espectrais para gráficos

ordem_espectral = {
    "O": 0,
    "B": 1,
    "A": 2,
    "F": 3,
    "G": 4,
    "K": 5,
    "M": 6
}

# Aplicação da ordem no dataframe

df_sem_ruido["ordem_espectral"] = df_sem_ruido["classe_espectral"].map(ordem_espectral)

In [ ]:
# Criação do histograma de classes espectrais por cluster

hist_classe_spec = alt.Chart(df_sem_ruido).transform_joinaggregate(

    # Quantidade de dados de cada classe espectral por cluster
    total='count()',
    groupby=['clusters']

).transform_calculate(

    # Frequência relativa de cada classe espectral por cluster
    percent='1 / datum.total'

).mark_bar(size=60).encode( # barras com largura de 60 pixels

    # Eixo x: Clusters
    x=alt.X(
        "clusters:N",
        title="Clusters",
        axis=alt.Axis(labelAngle=0), # Nomes dos clusters na horizontal
        scale=alt.Scale(paddingInner=0.6) # Aumenta o espaço entre as barras
    ),

    # Eixo y: Frequência 
    y=alt.Y(
        "sum(percent):Q", # soma os valores de percent (ou seja, 100%)
        stack="zero",
        title="Frequência (%)",
        axis=alt.Axis(format="%") # formata os valores de frequência relativa no formato de '%'
    ),

    # Colunas empilhadas decrescentemente pela ordem espectral definida anteriormente
    # Ou seja, O acima de B, B acima de A e assim por diante

    order=alt.Order(
    "ordem_espectral:Q",
    sort="descending" # ordenação decrescente
    ),

    # Colore as seções da barra de acordo com a classe espectral 
    color=alt.Color(
        "classe_espectral:N",
        title="Classes Espectrais",
        scale=alt.Scale(

            # Define a ordem das categorias na legenda (seguindo a ordem do empilhamento)
            domain=["O","B","A","F","G","K","M"], 

            # Mapeia cada categoria pelo dicionário cores_espectrais definido anteriormente
            range=[cores_espectrais[c] for c in ["O","B","A","F","G","K","M"]]
        )),

    # Define as informações a serem exibidas na 'caixinha flutuante'
    # Exibe classe espectral, cluster, frequência relativa e frequência absoluta de cada seçào em relação ao seu cluster

    tooltip=[
        alt.Tooltip("classe_espectral:N", title="Classe Espectral"),
        alt.Tooltip("clusters:Q", title="Cluster"),
        alt.Tooltip("sum(percent):Q", format=".2%", title="Frequência Relativa"),
        alt.Tooltip("count():Q", title="Frequência Absoluta")
    ]

    ).properties(
        height=450,
        width=500,
        title="Histograma de Classes Espectrais por Clusters"
    ).interactive()

In [ ]:
hist_classe_spec

Pelo histograma, observa-se que:
- Cluster 0: alta diversidade, destaque para F, seguida de A e G;
- Cluster 1: predominância de K, mas também tem M;
- Cluster 2: concentram-se principalmente nas classes A e F.

Com base nas classes observadas por cluster, nota-se que, de fato, o espectro das anãs brancas está próximo ao branco, o das gigantes vermelhas está próximo do vermelho, enquanto a sequência principal possui alta variabilidade.

Vale ressaltar que teoricamente anãs brancas têm classificação separada (D), mas o diagrama acima serve para comparar com as cores das classes espectrais padrão, indicando faixa de temperatura e cor.

A seguir, é definida uma função genérica para geração de boxplots por cluster, permitindo uma comparação de diferentes parâmetros físicos entre as populações.

In [ ]:
# Criação da função para exibir boxplots

def boxplot_por_cluster(dados, # dataframe a ser usado
    var_y, nome_y, # variável que será estudada no boxplot e seu nome
    titulo, # título do gráfico
    inv_y=False, # inverte o eixo y (por padrão, False)
    log_y=False): # eixo y em escala log (por padrão, False)

    # Cores de cada boxplot (ou seja, boxplot de cada cluster)

    lista = ["G", "M", "A"] # seleção das classes cujas cores se referirão aos boxplots
    cores = [cores_espectrais[classe] for classe in lista] # cores para os clusters 0, 1 e 2, respectivamente

    if log_y:
        # Eixo y na escala log e invertido, se especificado
        escala_y = alt.Scale(type="log", reverse=inv_y)
    else:
        # Inverte o eixo y se especificado
        escala_y = alt.Scale(reverse=inv_y)

    # Criação do boxplot com o dataframe dados
    box = alt.Chart(dados).mark_boxplot(size=60).encode( # caixa central com largura de 60 pixels

        # Eixo X: Clusters 
        x=alt.X(
            "clusters:N",
            title="Cluster",
            axis=alt.Axis(labelAngle=0) # Nomes dos clusters na horizontal
        ),

        # Eixo y: Variável quantitativa definida no parâmetro var_y
        y=alt.Y(
            f"{var_y}:Q",
            title=nome_y,
            scale=escala_y # escala_y definida no bloco if/else anterior
        ),

        # Cor baseada no cluster (definido anteriormente) tratado como Ordinal
        color=alt.Color(
            "clusters:O",
            title="Cluster",
            scale=alt.Scale(range=cores), # utiliza as cores definidas anteriormente
            legend=None # remove a legenda lateral (eixo X já identifica os clusters)
        ),

        tooltip=[
            alt.Tooltip(f"{var_y}:Q", title=nome_y) # título da variável tal qual definido pelo usuário
        ]
    ).properties(
        width=500,
        height=450,
        title=f"{titulo}" # título tal qual definido pelo usuário
    )
    
    return box

Abaixo, são construídos e exibidos boxplots para temperatura, magnitude absoluta, distância e raio, a fim de avaliar diferenças físicas entre os diferentes grupos estelares.

In [ ]:
# Criação dos boxplots de temperatura, magnitude absoluta, distância, raio e raio em escala log

boxplot_temp = boxplot_por_cluster(df_sem_ruido, "temp", "Temperatura (K)", "Distribuição de Temperaturas por Cluster")
boxplot_absmag = boxplot_por_cluster(df_sem_ruido, "absmag", "Magnitude Absoluta", "Distribuição de Magnitudes Absolutas por Cluster", inv_y=True)
boxplot_dist = boxplot_por_cluster(df_sem_ruido, "dist", "Distância (pc)", "Distribuição de Distâncias por Cluster")
boxplot_raio = boxplot_por_cluster(df_sem_ruido, "radius_est", "Raio (x Raio Solar)", "Distribuição de Raios por Cluster", log_y=False)
boxplot_raio_log = boxplot_por_cluster(df_sem_ruido, "radius_est", "Raio (x Raio Solar)", "Distribuição de Raios por Cluster (escala log)", log_y=True)

# Abaixo, seguem as exibições desses gráficos

In [ ]:
boxplot_temp

- Cluster 0: mediana em 6630 K, com alta dispersão (Q1 em 6088 K, Q3 em 7896 K)
- Cluster 1: mediana em 4461 K, com baixa dispersão (Q1 em 4075 K, Q3 em 4705 K)
- Cluster 2: mediana em 7938 K, com dispersão moderada (Q1 em 7307 K, Q3 em 8748 K)

Assim, observa-se que as temperaturas condizem com o comportamento dos grupos, e observa-se também que a sequência principal possui alta variabilidade de temperaturas, como esperado fisicamente.

In [ ]:
boxplot_absmag

Observa-se que as gigantes vermelhas possuem baixo valor de magnitude, indicando alta luminosidade, enquanto as anãs brancas possuem baixa luminosidade. Além disso, as estrelas da sequência principal possuem luminosidades intermediárias, sendo menores que as gigantes vermelhas, além de um pouco mais variantes, o que é compatível com a ampla faixa de estágios evolutivos presentes na população.

In [ ]:
boxplot_raio

In [ ]:
boxplot_raio_log

Nos boxplots de raio, é possível observar que as estrelas do cluster 1 apresentam raios significativamente maiores que os demais grupos, justificando sua caracterização como gigantes, enquanto o cluster 2 (anãs brancas) concentra os menores valores de raio. 

Na escala linear, a grande diferença de ordem de grandeza dificulta a visualização adequada das anãs brancas, tornando sua distribuição pouco perceptível, por isso, para esses dados, recomenda-se o uso da escala logarítmica para o eixo vertical.

In [ ]:
boxplot_dist

A limitação de distância observada para o cluster 2 evidencia um forte viés de detecção, associado à sua baixa magnitude absoluta e seus pequenos raios. Esse efeito compromete a representatividade dessa população para datasets com maiores distâncias, motivando a aplicação de um recorte amostral mais rigoroso, apresentado na etapa final do trabalho.

# Construção do Diagrama HR

Para a construção do diagrama HR, selecionou-se uma amostra aleatória contendo 10.000 estrelas, com `random_state=56` a fim de garantir reprodutibilidade. A construção é feita de forma gradual, de modo a explorar as ferramentas de visualização empregadas.

In [ ]:
# Amostra de 10000 pontos para a criação do diagrama HR final

dfp = df.sample(10000, random_state=56)

## HDBSCAN

### Gráfico 01 - Tamanho dependente do Raio

Inicialmente, define-se o tamanho dos pontos como sendo proporcionais ao raio estelar, desconsiderando ouliers fora do intervalo de 5% a 95% da distribuição de raios e normalização para um melhor contraste visual.

In [ ]:
# Definição da escala de tamanho dos pontos plotados com base no raio

r = dfp[col_raio] # extração da coluna de raios do dataframe

# Corta outliers para determinação do tamanho a fim de que não prejudicar a escala
r_clip = r.clip(r.quantile(0.05), r.quantile(0.95)) # achata os extremos

# Normalização do tamanho para visualização
s_min, s_max = 5, 60 # limites visuais do tamanho

# Aplicação de Min-Max Scaling
s = s_min + (r_clip - r_clip.min())/(r_clip.max() - r_clip.min()) * (s_max - s_min)

As cores dos pontos são atribuídas de acordo com os clusters definidos pelo HDBSCAN e do cluster adicional das anãs brancas, analisado separadamente, enquanto pontos considerados como ruído são exibidos em cinza.

In [ ]:
# Definição de cores para o plot de cada cluster

cores = {
    -1: "gray",  # ruído
     0: "yellow",
     1: "red",
     2: "blue"
}

c = dfp["clusters"].map(cores) # mapeamento de cores para cada classificação

O diagrama é construído com temperatura no eixo horizontal, em escala logarítmica e invertida, e magnitude absoluta no eixo vertical, também invertida, escolhas feitas de modo a respeitar a convenção clássica para o diagrama HR.

In [ ]:
# Plot do gráfico 01 da versão final do Diagrama HR

plt.style.use("default") # evitar que a imagem fique com fundo preto

fig, ax = plt.subplots(figsize=(10, 6)) # dimensões da figura

# Plot dos dados
plt.scatter (dfp[col_temp], dfp[col_mag], 
             s=s, # tamanho definido pela escala feita anteriormente
             c=c) # cor de acordo com o cluster

# Ajustes dos Eixos

plt.gca().invert_yaxis() # inverter eixo y
plt.gca().invert_xaxis() # inverter eixo x

plt.xscale("log") # eixo x na escala log para melhor visualização

plt.xlabel("Temperatura (K)") # eixo x equivalente ao eixo da temperatura
plt.ylabel("Magnitude Absoluta") # eixo y equivalente ao eixo da magnitude absoluta

# Título e exibição

plt.title(f"Diagrama HR com Clustering por HDBSCAN com alterações")

plt.show()

### Gráfico 02 - Adição de Contornos

Abaixo, é definida uma função para exibir elipses, com base na matriz de covariância dos clusters, a fim de repreentar sua dispersão estatística no espaço do diagrama.

In [ ]:
# Função para desenhar Elipse de Covariância

def desenhar_elipse(x, y, 
                    ax, # eixo do matplotlib
                    n_std=3.5, # quantos desvios padrões a elipse cobre
                    **kwargs): # argumentos extras
    
    cov = np.cov(x, y) # calcula a matriz de covariância
    mean_x, mean_y = np.mean(x), np.mean(y) # centro da elipse na média dos pontos

    # Extração dos autovalores e autovetores da matriz de cov
    # Autovetores indicam a direção de alinhamento do sistema de coordenadas com os clusters
    vals, vecs = np.linalg.eigh(cov)
    
    # Garantia de que o maior autovalor seja o primeiro (o de maior espalhamento)
    order = vals.argsort()[::-1] # índices do array vals em ordem decrescente
    vals, vecs = vals[order], vecs[:, order] # ordena vals e vecs decrescentemente

    # Ângulo da elipse
    # Calcula, em graus, o ângulo do vetor que aponta para a direção mais longa do cluster
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    # Largura e altura (raiz dos autovalores, ou seja, desvio padrão na direção pedida)
    width, height = 2 * n_std * np.sqrt(vals) # multiplica por n_std para controlar o tamanho da elipse

    ellipse = Ellipse((mean_x, mean_y), # centro da elipse
                      width=width, height=height, # largura e altura
                      angle=theta, # inclinação
                      fill=False, # sem cor de preenchimento
                      **kwargs) # adições extras

    ax.add_patch(ellipse) # adição da elipse

A seguir, são adicionadas as elipses aos clusters (exceto ao cluster de ruído), a fim de destacar suas posições no diagrama.

In [ ]:
# Plot do gráfico 02 da versão final do Diagrama HR

plt.style.use("default") # evitar fundo preto

fig, ax = plt.subplots(figsize=(10, 6)) # dimensões da figura

# Plot dos dados
plt.scatter (dfp[col_temp], dfp[col_mag], 
             s=s, # tamanho definido pela escala feita anteriormente
             c=c) # cor de acordo com o cluster

for cl in sorted(dfp["clusters"].unique()):
    if cl == -1:
        continue  # ignora ruído para a formação das elipses de contorno

    subset = dfp[dfp["clusters"] == cl] # parte do dataframe com todos os elementos do cluster cl

    # Desenho da elipse usando a função acima
    desenhar_elipse(subset[col_temp], subset[col_mag], ax,
                    n_std=2.1, edgecolor="black", linewidth=2)

# Ajustes gerais dos eixos

plt.gca().invert_yaxis() # inverter eixo y
plt.gca().invert_xaxis() # inverter eixo x

plt.xscale("log") # eixo x em escala log

plt.xlabel("Temperatura (K)") # eixo x equivalente ao eixo da temperatura
plt.ylabel("Magnitude Absoluta") # eixo y equivalente ao eixo da magnitude absoluta

# Título e plot do gráfico
plt.title(f"Diagrama HR com Clustering por HDBSCAN com alterações)")

plt.show()

### Gráfico 03 - Classes Espectrais

Agora, as estrelas passam a ser coloridas de acordo com sua classe espectral, a fim de aproximar a visualização do diagrama HR tradicional.

In [ ]:
# Criação de uma coluna que mapeia cores espectrais para cada classe espectral

dfp["cor_espectral"] = dfp["classe_espectral"].map(cores_espectrais)

O uso de fundo escuro auxilia no contraste visual, destacando as cores espectrais e adotando a estética usual para representações astronômicas. Além disso, é adicionado um eixo superior indicando as classes espectrais e suas cores.

In [ ]:
# Plot do gráfico 03 da versão final do Diagrama HR

plt.style.use("dark_background") # fundo preto

fig, ax = plt.subplots(figsize=(10, 6)) # dimensões da figura

# Plot dos dados
plt.scatter (dfp[col_temp], dfp[col_mag], 
             s=s, alpha=1, 
             c=dfp["cor_espectral"]) # cor de acordo com a classe espectral

for cl in sorted(dfp["clusters"].unique()):
    if cl == -1:
        continue  # ignora o ruído para a criação de elipses

    subset = dfp[dfp["clusters"] == cl]

    # Desenhar elipses brancas para destacar em relação ao fundo
    desenhar_elipse(subset[col_temp], subset[col_mag], ax,
                    n_std=2.1, edgecolor="white", linewidth=2)

# Ajustes dos eixos

plt.gca().invert_yaxis() # inverter eixo y
plt.gca().invert_xaxis() # inverter eixo x

plt.xscale("log") # eixo x na escala log para melhor visualização

plt.xlabel("Temperatura (K)") # eixo x equivalente ao eixo da temperatura
plt.ylabel("Magnitude Absoluta") # eixo y equivalente ao eixo da magnitude absoluta

# Adição de um eixo superior com as classes espectrais coloridas

# Criação do eixo x superior
ax_top = ax.secondary_xaxis('top')

# Rótulos das classes espectrais e as posições de seus respectivos ticks no eixo x superior
ticks_temp = [40000, 20000, 8500, 6750, 5600, 4500, 3000]
labels = ["O", "B", "A", "F", "G", "K", "M"]

# Inserção desses ticks com seus rótulos e posições
ax_top.set_xticks(ticks_temp)
ax_top.set_xticklabels(labels)

# Coloração os rótulos
for tick, label in zip(ax_top.get_xticklabels(), labels):
    tick.set_color(cores_espectrais[label]) # para cada tick, obtém sua cor espectral correspondente e o colore
    tick.set_fontsize(12) # fonte tamanho 12
    tick.set_fontweight("bold") # ticks em negrito

ax_top.minorticks_off() # sem minor ticks para evitar poluição visual

# Título e exibição do gráfico

plt.title(f"Diagrama HR com Clustering por HDBSCAN com alterações")

plt.show()

### Gráfico 04 - Retas com Raios

Por fim, são adicionadas linhas para raios constantes ao diagrama HR, deduzidas a partir da relação entre luminosidade, temperatura e raio estelar utilizada anteriormente, considerando o Sol como referência.

In [ ]:
# Plot do gráfico 04 (e último) da versão final do Diagrama HR

plt.style.use("dark_background") # fundo preto

fig, ax = plt.subplots(figsize=(10, 6)) # dimensões da figura

# Plot dos dados
plt.scatter (dfp[col_temp], dfp[col_mag], 
             s=s, alpha=1, 
             c=dfp["cor_espectral"]) # cor de acordo com a classe espectral

for cl in sorted(dfp["clusters"].unique()):
    if cl == -1:
        continue  # ignora o ruído para a criação de elipses

    subset = dfp[dfp["clusters"] == cl]

    # Desenhar elipses brancas para destacar em relação ao fundo
    desenhar_elipse(subset[col_temp], subset[col_mag], ax,
                    n_std=2.1, edgecolor="white", linewidth=2)

# Criação das linhas tracejadas que acompanham os valores de raio em unidades solares

# Valores referentes ao Sol
T_sun = 5777 # temperatura (K)
M_sun = 4.83 # magnitude absoluta

# Cria 500 pontos de temperatura nos intervalos do gráfico
# np.logspace para distribuir os pontos de acordo com a escala log do eixo x
T_vals = np.logspace(np.log10(3000), np.log10(21500), 500)

# Lista de raios em unidades solares cujas linhas tracejadas serão plotadas
raios = [0.01, 0.05, 0.1, 1, 5, 10, 50, 100]

# Cálculo dos valores de magnitude para cada R em relação aos pontos gerados em T_vals
for R in raios:
    M_vals = (
        M_sun
        - 5*np.log10(R)
        - 10*np.log10(T_vals / T_sun)
    )

    # Plot das linhas tracejadas
    ax.plot(T_vals, M_vals, linestyle="--", linewidth=1, 
            color="white", alpha=0.5) # semitransparentes para não ofuscar os pontos das estrelas

    # Inserção do rótulo de cada linha
    ax.text(

        # Coordenadas do texto
        T_vals[-1], M_vals[-1], # texto na esquerda, próximo ao início da linha
        f"{R} R☉", # R como valor do raio em unidades solares
        fontsize=9,
        verticalalignment='bottom' # base das letras tocando o ponto
    )

# Ajustes dos eixos

plt.gca().invert_yaxis() # inverter eixo y
plt.gca().invert_xaxis() # inverter eixo x

plt.xscale("log") # eixo x na escala log para melhor visualização

plt.xlabel("Temperatura (K)") # eixo x equivalente ao eixo da temperatura
plt.ylabel("Magnitude Absoluta") # eixo y equivalente ao eixo da magnitude absoluta

# Adição de um eixo superior com as classes espectrais coloridas

# Criação do eixo x superior
ax_top = ax.secondary_xaxis('top')

# Rótulos das classes espectrais e as posições de seus respectivos ticks no eixo x superior
ticks_temp = [40000, 20000, 8500, 6750, 5600, 4500, 3000]
labels = ["O", "B", "A", "F", "G", "K", "M"]

# Inserção desses ticks com seus rótulos e posições
ax_top.set_xticks(ticks_temp)
ax_top.set_xticklabels(labels)

# Coloração os rótulos
for tick, label in zip(ax_top.get_xticklabels(), labels):
    tick.set_color(cores_espectrais[label]) # para cada tick, obtém sua cor espectral correspondente e o colore
    tick.set_fontsize(12) # fonte tamanho 12
    tick.set_fontweight("bold") # ticks em negrito

ax_top.minorticks_off() # sem minor ticks para evitar poluição visual

# Título e exibição do gráfico

plt.title(f"Diagrama HR com Clustering por HDBSCAN com alterações")

plt.show()

# Limitação de Distância e Viés de Tipo Estelar

A análise anterior evidenciou um viés observacional significativo relacionado à distância, impactando na baixa representatividade de anãs brancas, por sua baixa luminosidade, impondo um limite de distância menor para os instrumentos utilizados na detecção. Nesta seção, será investigado como tal limitação afeta a distribuição dos tipos estelares e, consequentemente, o desempenho da clusterização.

Primeiramente, é recriado um conjunto de dados a partir da leitura original. Em seguida, aplica-se um filtro de distância, garantindo que todas as distâncias estejam abaixo de 50 parsecs, e exibe-se um novo diagrama, evidenciando as alterações na proporção entre populações estelares.

In [ ]:
# Criação do dataframe em que será aplicado o filtro de distância
df_dist = leitura.copy().drop(columns=cols_remover, errors="ignore")

# Inspeção das características do dataframe
print("Dimensões do dataframe:", df_dist.shape)
df_dist.info()
df_dist.head()

In [ ]:
# Aplicação do filtro de distância

df_dist = df_dist[df_dist['dist'] < 50]
df_dist.describe()

In [ ]:
# Visualização inicial dos dados com distância reduzida

plt.style.use("default") # garantir que o fundo será branco (evita interferência de outros gráficos)

plt.figure(figsize=(8, 6)) # definição das dimensões da figura

plt.scatter(df_dist[col_color], df_dist[col_mag], c='blue', s=2) # plot de mag abs por index de cor

plt.gca().invert_yaxis() # inversão do eixo y para seguir o padrão do diagrama HR

# Definição do nome dos eixos e do título do gráfico
plt.xlabel("Color Index (ci)")
plt.ylabel("Magnitude Absoluta")
plt.title("Diagrama HR - Dados com distância reduzida")

plt.show() # exibição do gráfico

Antes da reconstrução do diagrama e das análises, é realizada novamente a limpeza dos dados e o cálculo de parâmetros físicos.

In [ ]:
# Limpeza dos dados
df_dist = df_dist.replace([np.inf, -np.inf], np.nan) # transformação de possíveis valores infinitos em NaN
df_dist = df_dist.dropna(subset=[col_color, col_mag, col_temp, col_lum]) # remoção de entradas com valores NaN nas principais colunas

# Filtros para consistência e estabilidade de clustering
df_dist = df_dist[df_dist[col_color].between(-0.5, 2.5)]
df_dist = df_dist[df_dist[col_mag].between(-15, 20)]
df_dist = df_dist[df_dist[col_temp].between(2000, 50000)]
df_dist = df_dist[df_dist[col_lum] > 0]

# Definição das classes espectrais
classes = [0, 3700, 5200, 6000, 7500, 10000, 30000, np.inf] # valores limites de cada classe
labels = ["M", "K", "G", "F", "A", "B", "O"] # nome de cada classe

# Cores que se referem a cada classe para futuros plots
cores_espectrais = { 
    "O": "#4f6cff", 
    "B": "#7ea8ff", 
    "A": "#dbe9ff", 
    "F": "#fff7d6", 
    "G": "#ffe86b", 
    "K": "#ffb347", 
    "M": "#ff5c5c" 
}

# Criação da coluna de raios em unidades solares
df_dist["radius_est"] = np.sqrt(df_dist[col_lum] / ((df_dist[col_temp] / Tsun) ** 4))

# Criação de uma coluna que associa à cada entrada, sua classe espectral
df_dist["classe_espectral"] = pd.cut(df_dist[col_temp], bins=classes, labels=labels, right=False)

# Criação da coluna de log(temp) de cada estrela
df_dist["log_temp"] = np.log10(df_dist["temp"])

No bloco a seguir, novamente são selecionadas a magnitude absoluta e o logaritmo da temperatura efeita, e também realizada a padronização via Z-score, a fim de garantir equilíbrio estatístico, evitando um efeito artificial causado por diferenças de escala.

In [ ]:
# Seleção das variáveis Magnitude Abs. e Log da Temp. para clustering
variaveis2 = df_dist[[col_mag, col_temp_log]] # 

# Obtenção do Z-score das variáveis
scaler = StandardScaler() # criação da instância que irá definir a nova escala das variáveis
variaveis_s2 = scaler.fit_transform(variaveis2) # aplicação no conjunto de variáveis

Agora, aplica-se o algoritmo HDBSCAN à nova amostra. Dessa forma, mudanças na separação entre os grupos devem passar a refletir a distribuição física real de forma mais precisa.

In [ ]:
# Aplicação do modelo de clustering de HDBSCAN

min_cluster_size = 65 # tamanho mínimo dos clusters p/ manter estruturas maiores
cluster_hdbscan = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size) # aplicação do modelo

df_dist["clusters"] = cluster_hdbscan.fit_predict(variaveis_s2) # armazenamento das classificações

No diagrama gerado abaixo, é possível observar que para o conjunto de dados limitados por distância, o algoritmo HDBSCAN foi capaz de identificar os clusters relativos às anãs brancas, às gigantes e à sequência principal, alcançando inclusive a cauda fina da sequência principal.

In [ ]:
# Visualização dos clusters
plot_cluster(df_dist[col_temp], df_dist[col_mag], df_dist["clusters"], "HDBSCAN")

Em seguida, é reconstruído o histograma da distribuição relativa das classes espectrais por cluster, observando estrelas sem ruído e com limitação em distância.

Os clusters ainda apresentam uma distribuição fisicamente coerente, mesmo com o corte em distância, de modo a refletir a composição usual da vizinhança solar.

In [ ]:
# Definição de ordem para as classes espectrais para gráficos
ordem_espectral = {
    "O": 0,
    "B": 1,
    "A": 2,
    "F": 3,
    "G": 4,
    "K": 5,
    "M": 6
}

# Criação de um df sem ruído para as análises a seguir
df_dist_sem_ruido = df_dist[df_dist["clusters"] != -1].copy()

# Aplicação da ordem no dataframe
df_dist_sem_ruido["ordem_espectral"] = df_dist_sem_ruido["classe_espectral"].map(ordem_espectral)

In [ ]:
# Criação do histograma de classes espectrais por cluster

hist_classe_spec2 = alt.Chart(df_dist_sem_ruido).transform_joinaggregate(

    # Quantidade de dados de cada classe espectral por cluster
    total='count()',
    groupby=['clusters']

).transform_calculate(

    # Frequência relativa de cada classe espectral por cluster
    percent='1 / datum.total'

).mark_bar(size=60).encode( # barras com largura de 60 pixels

    # Eixo x: Clusters
    x=alt.X(
        "clusters:N",
        title="Clusters",
        axis=alt.Axis(labelAngle=0), # Nomes dos clusters na horizontal
        scale=alt.Scale(paddingInner=0.6) # Aumenta o espaço entre as barras
    ),

    # Eixo y: Frequência 
    y=alt.Y(
        "sum(percent):Q", # soma os valores de percent (ou seja, 100%)
        stack="zero",
        title="Frequência (%)",
        axis=alt.Axis(format="%") # formata os valores de frequência relativa no formato de '%'
    ),

    # Colunas empilhadas decrescentemente pela ordem espectral definida anteriormente
    # Ou seja, O acima de B, B acima de A e assim por diante

    order=alt.Order(
    "ordem_espectral:Q",
    sort="descending" # ordenação decrescente
    ),

    # Colore as seções da barra de acordo com a classe espectral 
    color=alt.Color(
        "classe_espectral:N",
        title="Classes Espectrais",
        scale=alt.Scale(

            # Define a ordem das categorias na legenda (seguindo a ordem do empilhamento)
            domain=["O","B","A","F","G","K","M"], 

            # Mapeia cada categoria pelo dicionário cores_espectrais definido anteriormente
            range=[cores_espectrais[c] for c in ["O","B","A","F","G","K","M"]]
        )),

    # Define as informações a serem exibidas na 'caixinha flutuante'
    # Exibe classe espectral, cluster, frequência relativa e frequência absoluta de cada seçào em relação ao seu cluster

    tooltip=[
        alt.Tooltip("classe_espectral:N", title="Classe Espectral"),
        alt.Tooltip("clusters:Q", title="Cluster"),
        alt.Tooltip("sum(percent):Q", format=".2%", title="Frequência Relativa"),
        alt.Tooltip("count():Q", title="Frequência Absoluta")
    ]

    ).properties(
        height=450,
        width=500,
        title="Histograma de Classes Espectrais por Clusters"
    ).interactive()

In [ ]:
hist_classe_spec2

Observa-se que o cluster 0 corresponte às anãs brancas, o cluster 1 às gigantes e o cluster 2 à sequência principal.

A maior mudança ocorreu na sequência principal, que agora possui um deslocamento para cores mais avermelhadas, por incluir a cauda fina, composta majoritariamente por estrelas pouco luminosas e mais frias. Esse efeito é consistente com o recorte em distância, pois é proveniente do favorecimento de estrelas com menor magnitude absoluta, pouco visíveis à grandes distâncias.

Nesta etapa, replica-se a função para boxplot com pequenos ajustes, a fim de realizar novamente a análise das propriedades físicas por cluster. 

Serão também replicados quatro boxplots principais, optando pela exclusão apenas do raio em escala linear, por sua ampla dispersão, de modo que a escala logarítmica se mostra mais adequada para a análise.

In [ ]:
# Criação da função para exibir boxplots

def boxplot_por_cluster2(dados, # dataframe a ser usado
    var_y, nome_y, # variável que será estudada no boxplot e seu nome
    titulo, # título do gráfico
    inv_y=False, # inverte o eixo y (por padrão, False)
    log_y=False): # eixo y em escala log (por padrão, False)

    # Cores de cada boxplot (ou seja, boxplot de cada cluster)

    lista = ["A", "M", "G"] # seleção das classes cujas cores se referirão aos boxplots
    cores = [cores_espectrais[classe] for classe in lista] # cores para os clusters 0, 1 e 2, respectivamente

    if log_y:
        # Eixo y na escala log e invertido, se especificado
        escala_y = alt.Scale(type="log", reverse=inv_y)
    else:
        # Inverte o eixo y se especificado
        escala_y = alt.Scale(reverse=inv_y)

    # Criação do boxplot com o dataframe dados
    box = alt.Chart(dados).mark_boxplot(size=60).encode( # caixa central com largura de 60 pixels

        # Eixo X: Clusters 
        x=alt.X(
            "clusters:N",
            title="Cluster",
            axis=alt.Axis(labelAngle=0) # Nomes dos clusters na horizontal
        ),

        # Eixo y: Variável quantitativa definida no parâmetro var_y
        y=alt.Y(
            f"{var_y}:Q",
            title=nome_y,
            scale=escala_y # escala_y definida no bloco if/else anterior
        ),

        # Cor baseada no cluster (definido anteriormente) tratado como Ordinal
        color=alt.Color(
            "clusters:O",
            title="Cluster",
            scale=alt.Scale(range=cores), # utiliza as cores definidas anteriormente
            legend=None # remove a legenda lateral (eixo X já identifica os clusters)
        ),

        tooltip=[
            alt.Tooltip(f"{var_y}:Q", title=nome_y) # título da variável tal qual definido pelo usuário
        ]
    ).properties(
        width=500,
        height=450,
        title=f"{titulo}" # título tal qual definido pelo usuário
    )
    
    return box

In [ ]:
# Criação dos boxplots de temperatura, magnitude absoluta, distância, raio e raio em escala log

boxplot_temp2 = boxplot_por_cluster2(df_dist_sem_ruido, "temp", "Temperatura (K)", "Distribuição de Temperaturas por Cluster")
boxplot_absmag2 = boxplot_por_cluster2(df_dist_sem_ruido, "absmag", "Magnitude Absoluta", "Distribuição de Magnitudes Absolutas por Cluster", inv_y=True)
boxplot_dist2 = boxplot_por_cluster2(df_dist_sem_ruido, "dist", "Distância (pc)", "Distribuição de Distâncias por Cluster")
boxplot_raio_log2 = boxplot_por_cluster2(df_dist_sem_ruido, "radius_est", "Raio (x Raio Solar)", "Distribuição de Raios por Cluster", log_y=True)

# Abaixo, seguem as exibições desses gráficos

In [ ]:
boxplot_temp2

Para o novo recorte por distância, observou-se os seguintes valores:
- Cluster 0: mediana em 7276 K, com dispersão moderada (Q1 em 5853 K, Q3 em 8839 K)
- Cluster 1: mediana em 4715 K, com baixa dispersão (Q1 em 4422 K, Q3 em 4964 K)
- Cluster 2: mediana em 5070 K, com alta dispersão (Q1 em 3222 K, Q3 em 9256 K)

Nota-se que a maior mudança ocorreu no cluster 2, que representa a sequência principal, possuindo uma amplitude ainda maior de valores devido à inclusão da cauda fina.

In [ ]:
boxplot_absmag2

Para a magnitude absoluta, observa-se novamente que as gigantes vermelhas possuem baixo valor de magnitude, indicando alta luminosidade, enquanto as anãs brancas possuem baixa luminosidade.

Novamente, as estrelas da sequência principal possuem luminosidades intermediárias, sendo menores que as gigantes vermelhas, mas a mediana é mais baixa e a variação ainda maior, o que é compatível com a ampla faixa de estágios evolutivos presentes na população, e causado pela inclusão da cauda fina.

In [ ]:
boxplot_dist2

Mesmo com o recorte até 50 parsecs, a distribuição de distâncias mostra-se agora mais consistente com a vizinhança solar. É possível notar que as anãs brancas concentram-se majoritariamente abaixo de cerca de 25 parsecs, enquanto outros grupos se estendem até valores próximos do limite. 

Observa-se também a baixa presença de gigantes vermelhas nas regiões mais próximas, enquanto as estrelas da sequência principal aparecem distribuídas ao longo de praticamente todo o intervalo de distâncias analisado.

In [ ]:
boxplot_raio_log2

A distribuição de raios segue o padrão esperado, de modo que estrelas da sequência principal estão na ordem de $1\,R_\odot$, anãs brancas possuem raios da ordem de $0.01\,R_\odot$ e gigantes estão próximas de $10\,R_\odot$, o que reforça as diferenças físicas entre os grupos, assim como suas classificações, em especial nos grupos de anãs brancas e gigantes.

Por fim, é recriado o diagrama HR final, incluindo classificação espectral, visualização do tamanho proporcional aos raios.

Além disso, as curvas para raios constantes reforçam a interpretação dos clusters e sua divisão em anãs brancas, sequência principal e gigantes, mostrando que as estrelas ocupam regiões distintas não só em luminosidade e temperatura, mas também em escala física real.

In [ ]:
# Definição da escala de tamanho dos pontos plotados com base no raio

r = df_dist[col_raio] # extração da coluna de raios do dataframe

# Corta outliers para determinação do tamanho a fim de que não prejudicar a escala
r_clip = r.clip(r.quantile(0.05), r.quantile(0.95)) # achata os extremos

# Normalização do tamanho para visualização
s_min, s_max = 5, 60 # limites visuais do tamanho

# Aplicação de Min-Max Scaling
s = s_min + (r_clip - r_clip.min())/(r_clip.max() - r_clip.min()) * (s_max - s_min)

c = df_dist["clusters"].map(cores) # mapeamento de cores para cada classificação

# Criação de uma coluna que mapeia cores espectrais para cada classe espectral
df_dist["cor_espectral"] = df_dist["classe_espectral"].map(cores_espectrais) 

In [ ]:
# Plot do gráfico 03 da versão final do Diagrama HR

plt.style.use("dark_background") # fundo preto

fig, ax = plt.subplots(figsize=(10, 6)) # dimensões da figura

# Plot dos dados
plt.scatter (df_dist[col_temp], df_dist[col_mag], 
             s=s, alpha=1, 
             c=df_dist["cor_espectral"]) # cor de acordo com a classe espectral

for cl in sorted(df_dist["clusters"].unique()):
    if cl == -1:
        continue  # ignora o ruído para a criação de elipses

    subset = df_dist[df_dist["clusters"] == cl]

    # Desenhar elipses brancas para destacar em relação ao fundo
    desenhar_elipse(subset[col_temp], subset[col_mag], ax,
                    n_std=2.1, edgecolor="white", linewidth=2)

# Criação das linhas tracejadas que acompanham os valores de raio em unidades solares

# Valores referentes ao Sol
T_sun = 5777 # temperatura (K)
M_sun = 4.83 # magnitude absoluta

# Cria 500 pontos de temperatura nos intervalos do gráfico
# np.logspace para distribuir os pontos de acordo com a escala log do eixo x
T_vals = np.logspace(np.log10(3000), np.log10(15000), 500)

# Lista de raios em unidades solares cujas linhas tracejadas serão plotadas
raios = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]

# Cálculo dos valores de magnitude para cada R em relação aos pontos gerados em T_vals
for R in raios:
    M_vals = (
        M_sun
        - 5*np.log10(R)
        - 10*np.log10(T_vals / T_sun)
    )

    # Plot das linhas tracejadas
    ax.plot(T_vals, M_vals, linestyle="--", linewidth=1, 
            color="white", alpha=0.5) # semitransparentes para não ofuscar os pontos das estrelas

    # Inserção do rótulo de cada linha
    ax.text(

        # Coordenadas do texto
        T_vals[-1], M_vals[-1], # texto na esquerda, próximo ao início da linha
        f"{R} R☉", # R como valor do raio em unidades solares
        fontsize=9,
        verticalalignment='bottom' # base das letras tocando o ponto
    )

# Ajustes dos eixos

plt.gca().invert_yaxis() # inverter eixo y
plt.gca().invert_xaxis() # inverter eixo x

plt.xscale("log") # eixo x na escala log para melhor visualização

plt.xlabel("Temperatura (K)") # eixo x equivalente ao eixo da temperatura
plt.ylabel("Magnitude Absoluta") # eixo y equivalente ao eixo da magnitude absoluta

# Adição de um eixo superior com as classes espectrais coloridas

# Criação do eixo x superior
ax_top = ax.secondary_xaxis('top')

# Rótulos das classes espectrais e as posições de seus respectivos ticks no eixo x superior
ticks_temp = [40000, 20000, 8500, 6750, 5600, 4500, 3000]
labels = ["O", "B", "A", "F", "G", "K", "M"]

# Inserção desses ticks com seus rótulos e posições
ax_top.set_xticks(ticks_temp)
ax_top.set_xticklabels(labels)

# Coloração os rótulos
for tick, label in zip(ax_top.get_xticklabels(), labels):
    tick.set_color(cores_espectrais[label]) # para cada tick, obtém sua cor espectral correspondente e o colore
    tick.set_fontsize(12) # fonte tamanho 12
    tick.set_fontweight("bold") # ticks em negrito

ax_top.minorticks_off() # sem minor ticks para evitar poluição visual

# Título e exibição do gráfico

plt.title(f"Diagrama HR com Clustering por HDBSCAN com alterações")

plt.show()

# Conclusões

Este trabalho demonstrou como técnicas de clustering podem ser aplicadas ao Diagrama HR para identificar populações estelares a partir de dados observacionais reais. A partir da extração e tratamento do dataset, foi possível construir representações visuais consistentes e explorar diferentes abordagens de agrupamento, analisando seus impactos na identificação de padrões físicos.

Os resultados evidenciaram que métodos de clusterização, como o HDBSCAN, são úteis na recuperação de estruturas fisicamente significativas, como anãs brancas, gigantes e estrelas da sequência principal, quando aplicados às variáveis fundamentais do diagrama (magnitude absoluta e temperatura). A correspondência entre os agrupamentos estatísticos e as regiões evolutivas esperadas confirma que a análise de dados pode, de fato, ser uma aliada na revelação de estruturas astrofísicas reais.

Entretanto, o processo também mostrou que técnicas de clustering não são suficientes por si só. A má separação inicial entre algumas classes estelares, especialmente entre sequência principal e gigantes, evidenciou que a interpretação dos resultados exige conhecimento teórico prévio da astrofísica estelar. Além disso, a escassez de dados de anãs brancas e o forte viés observacional em função da distância influenciaram diretamente a formação dos clusters.

A análise do recorte em distância foi fundamental para revelar o impacto do viés de magnitude na composição da amostra. Ao restringir o volume analisado, foi possível reduzir distorções na densidade relativa das populações e obter agrupamentos mais fisicamente coerentes. Esse ajuste exigiu não apenas conhecimento técnico em ciência de dados, mas também compreensão das limitações observacionais da astronomia.

O trabalho também destacou a importância da visualização científica. A construção progressiva dos diagramas, a exploração de diferentes escalas, a inclusão de curvas de raio constante e a análise por boxplots e histogramas foram essenciais para interpretar os resultados de forma integrada.

Assim, trata-se de um estudo interdisciplinar entre astrofísica e ciência de dados, no qual os algoritmos forneceram uma estrutura inicial de análise, mas a interpretação física foi indispensável para validar, ajustar e entender os resultados. Diferentes escolhas metodológicas produziram diferentes agrupamentos, reforçando que a clusterização depende tanto da natureza dos dados quanto do contexto físico.

Apesar do alto nível de ruído característico de dados astronômicos, foi possível identificar e corrigir vieses observacionais, alcançando resultados compatíveis com o conhecimento teórico esperado. Assim, o estudo demonstra que a combinação entre análise estatística, visualização e fundamentação física é essencial para extrair significado científico de grandes conjuntos de dados.

# Referências

O dataset utilizado foi obtido por meio do repositório disponibilizado no GitHub por Lamee, sendo originalmente baseado no HYG Stellar Database, mantido pelo Astronexus.

Parte da abordagem visual adotada foi inspirada no vídeo HR Diagram Introduction, de Jimmy Newland (2026).

- LAMEE, Adam. Stars dataset. CODINGinK12 Repository. GitHub. Disponível em: https://github.com/adamlamee/CODINGinK12. Acesso em: 21 fev. 2026.

- ASTRONEXUS. HYG Stellar Database. Disponível em: https://www.astronexus.com/projects/hyg. Acesso em: 21 fev. 2026.

- OLIVEIRA, Kepler de Souza; SARAIVA, Maria de Fátima Oliveira. Astronomia e astrofísica. 2. ed. São Paulo: Livraria da Física, 2014.

- CARROLL, Bradley W.; OSTLIE, Dale A. An Introduction to Modern Astrophysics. 2. ed. Cambridge: Cambridge University Press, 2017.

- SCIKIT-LEARN DEVELOPERS. Clustering and Evaluation - Scikit-Learn 1.x Documentation. Disponível em: https://scikit-learn.org/stable/modules/clustering.html . Acesso em: 08 fev. 2026.

- Material Disponibilizado na pasta da Semana 05: https://drive.google.com/file/d/1niDu1Fh8gQlDA_ezjMeNklUWf4MufKvk/view . Acesso em: 08 fev. 2026.

- NEWLAND, Jimmy. HR Diagram Introduction. YouTube. Disponível em: https://www.youtube.com/watch?v=58w9vJhZO6w. Acesso em: 17 fev. 2026.